In [1]:
from torchsummary import summary
from python_lib.modules import *
import torch
import numpy as np
import os
from python_lib.saveasfile import *

c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html


In [11]:
input_feats = torch.rand([1,2,64]).to("cuda")
input_feats_np = input_feats.cpu().detach().numpy()
input_feats_np = np.reshape(input_feats_np, (2,64))
print(input_feats_np)
dim = input_feats_np.shape

flatten_inputs = input_feats_np.flatten()
with open(os.path.join("ECAPAweights", f"initialinput.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_inputs.tobytes())

[[0.48793715 0.03895187 0.5718016  0.12249893 0.38095903 0.57180667
  0.6064527  0.6119571  0.06414747 0.76479465 0.45228392 0.29536617
  0.07002324 0.4701903  0.27933836 0.12131184 0.71218824 0.09092271
  0.8129148  0.29598218 0.46104175 0.7006878  0.40521938 0.27590483
  0.27556282 0.53625596 0.9908802  0.8078377  0.20478153 0.9662656
  0.71368986 0.92577356 0.8016781  0.00675195 0.03217447 0.08390254
  0.60445607 0.31654936 0.3509106  0.22730839 0.9643708  0.3126679
  0.2621218  0.00258678 0.9726101  0.10385472 0.1740588  0.15319914
  0.4173507  0.81994754 0.89946204 0.18791324 0.40089375 0.5389032
  0.16250598 0.55830073 0.18466216 0.32261306 0.06586313 0.3657987
  0.8261475  0.36473054 0.5412028  0.12367254]
 [0.09935516 0.69415635 0.5582605  0.41461217 0.2789762  0.86553675
  0.8209544  0.6945344  0.77635235 0.09968209 0.31738305 0.99634093
  0.44244152 0.01629257 0.33295602 0.1899671  0.8490492  0.7557361
  0.02923572 0.91277575 0.77060086 0.27032113 0.71409255 0.9110145
  0.853

In [5]:
model = ECAPA_TDNN(input_size = 2, channels=[8,8,8,8,16], lin_neurons=6, device = "cuda").to("cuda")

initialblock = BlockSave(model.blocks[0].return_layers(), "initialblock", "ECAPAweights")
initialblock.save()

seres2_1 = BlockSave(model.blocks[1].return_layers(), "seres2_1", "ECAPAweights")
seres2_1.save()
seres2_2 = BlockSave(model.blocks[2].return_layers(), "seres2_2", "ECAPAweights")
seres2_2.save()
seres2_3 = BlockSave(model.blocks[3].return_layers(), "seres2_3", "ECAPAweights")
seres2_3.save()

mfa = BlockSave(model.mfa.return_layers(), "mfa", "ECAPAweights")
mfa.save()

asp = BlockSave(model.asp.return_layers(), "asp", "ECAPAweights")
asp.save()

asp_bn = BlockSave(model.asp_bn.return_layers(), "asp_bn", "ECAPAweights")
asp_bn.save()

fc = BlockSave([model.final[1]], "fc", "ECAPAweights")
fc.save()

In [6]:
model.blocks[0].return_layers()

[Conv1d(2, 8, kernel_size=(5,), stride=(1,)),
 BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)]

In [7]:
ECAPA_weights = BlockSave(model.return_layers(), "ECAPA_TDNN", "ECAPAweights")
ECAPA_weights.save()

In [10]:
model.blocks[0](input_feats)

tensor([[[-3.9648e-01, -3.9648e-01,  3.4967e-01,  ..., -3.9648e-01,
          -3.9648e-01, -3.9648e-01],
         [ 5.2983e-01, -1.5384e+00, -9.3842e-01,  ..., -1.1564e-01,
           3.5418e-01,  1.2137e-01],
         [-1.0124e+00, -8.4545e-02,  3.4077e-01,  ..., -8.7633e-01,
           2.0481e+00, -1.0124e+00],
         ...,
         [-6.1792e-02, -6.1792e-02, -6.1792e-02,  ..., -6.1792e-02,
          -6.1792e-02, -6.1792e-02],
         [-1.5780e-01, -1.5780e-01, -1.5780e-01,  ..., -1.5780e-01,
          -1.5780e-01, -1.5780e-01],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]],

        [[-3.9648e-01, -3.9648e-01,  3.8759e-01,  ..., -3.9648e-01,
          -3.9648e-01, -3.9648e-01],
         [ 2.6586e+00, -2.6063e-01, -1.5384e+00,  ...,  4.8656e-01,
           7.4734e-01,  2.8400e-01],
         [ 1.3009e-01,  3.0361e+00, -1.0124e+00,  ...,  2.5610e-01,
           1.7261e+00, -6.7679e-01],
         ...,
         [ 7.4382e+00, -6